In [1]:
import pandas as pd
import regex as re
from collections import Counter
import emoji
from gensim.models.word2vec import Word2Vec
import nltk
from nltk import word_tokenize
from gensim.models import FastText
from gensim.models import KeyedVectors
#from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import numpy as np
#from tqdm import tqdm
from gensim.models.callbacks import CallbackAny2Vec
#import matplotlib.pyplot as plt
import keras
from keras.preprocessing.text import Tokenizer
from keras.utils.data_utils import pad_sequences
#from keras.initializers import Constant
from keras.layers import ReLU
from keras.layers import Dropout
from keras.models import Sequential
from keras.layers import Dense, Flatten, Embedding, Input, LSTM, Bidirectional, TimeDistributed, Conv1D, MaxPooling1D, GlobalMaxPooling1D
from keras.optimizers import Adam
from keras import Model
from keras.models import load_model
from keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
import math
nltk.download('punkt')

2023-04-26 11:07:49.350694: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-26 11:07:49.350726: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[nltk_data] Downloading package punkt to /home/user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
df1 = pd.read_pickle('dataset/hinglish_tweet_ids_part1.pkl')
print(df1.columns)
df1[['tweet_text']]

Index(['tweet_id', 'tweet_text', 'retweet_count', 'favorite_count',
       'user_name', 'user_id', 'user_description', 'user_verified',
       'user_followers_count', 'user_friends_count', 'user_listed_count',
       'user_favourites_count', 'user_statuses_count', 'user_created_at'],
      dtype='object')


,tweet_text
0,@ajuonline #sarcasm Haal chaal thik thak hai-s...
1,@I_Am_Hollywood - Charlie Haas? #sarcasm
2,Chetan Bhagat #epicmale #sarcasm
3,@Bo_jasim86 LOOL 7atheer affaa 3allaik rgoobti...
4,"@FENUSF tees2al 3anek el 3afiaa 7abeeb galbe ,..."
...,...
88586,@Abhi_4_Nation @narendramodi @PrakashJavdekar ...
88587,Weh pehle-humari-pocket-se-SIRF-THODA-SA-nikal...
88588,@rssurjewala Waqt ayega ki kuch janwar sirf zo...
88589,@LawBharath @LEDtvn Bakhton ko pata nahi hoga ...


In [3]:
df2 = pd.read_pickle('dataset/hinglish_tweet_ids_part2.pkl')
print(df2.columns)
df2[['tweet_text']]

Index(['tweet_id', 'tweet_text', 'retweet_count', 'favorite_count',
       'user_name', 'user_id', 'user_description', 'user_verified',
       'user_followers_count', 'user_friends_count', 'user_listed_count',
       'user_favourites_count', 'user_statuses_count', 'user_created_at'],
      dtype='object')


,tweet_text
0,@ajuonline #sarcasm Haal chaal thik thak hai-s...
1,@I_Am_Hollywood - Charlie Haas? #sarcasm
2,Chetan Bhagat #epicmale #sarcasm
3,@Bo_jasim86 LOOL 7atheer affaa 3allaik rgoobti...
4,"@FENUSF tees2al 3anek el 3afiaa 7abeeb galbe ,..."
...,...
105192,"""Kaali kaali khaali raaton se Hone lagi hai do..."
105193,Jao pehle usese jakar puchho jisne meri Jameen...
105194,"Hai tu agar mera dilbar, Hai tu agar mera dilb..."
105195,@iam_ashima nahi dekhna hai yaar mere KO khhon...


In [4]:
df3 = pd.read_pickle('dataset/hinglish_tweet_ids_part3.pkl')
print(df3.columns)
df3[['tweet_text']]

Index(['tweet_id', 'tweet_text', 'retweet_count', 'favorite_count',
       'user_name', 'user_id', 'user_description', 'user_verified',
       'user_followers_count', 'user_friends_count', 'user_listed_count',
       'user_favourites_count', 'user_statuses_count', 'user_created_at'],
      dtype='object')


,tweet_text
0,@muneebfaruq oho... tention na lo.. paanch saa...
1,@temmujin Buss mujhay bher aik choutee moutee ...
2,Pakhtoon K Karname yaad krogey to tmko sharam ...
3,@sana_virk wohi na ! zindagi main ek dafa us n...
4,"Paap kya punya kya tu bhula de, karma kar phal..."
...,...
7445,@Javedakhtarjadu salaam alecum sir. aapki izaj...
7446,"Khud Ko Khuda Samjane Wale, Tu Khuda Ka Khouf ..."
7447,"hoti nahi dosti kabhi chehre se, dosti to dil ..."
7448,Jab Jab Dil Dhadkta Hain Tab Tab Tumhari Yaad ...


In [5]:
df_hinglish = pd.concat([df1, df2, df3], ignore_index=True)
print(df_hinglish.columns)
print('--------------------')
print(df_hinglish)

Index(['tweet_id', 'tweet_text', 'retweet_count', 'favorite_count',
       'user_name', 'user_id', 'user_description', 'user_verified',
       'user_followers_count', 'user_friends_count', 'user_listed_count',
       'user_favourites_count', 'user_statuses_count', 'user_created_at'],
      dtype='object')
--------------------
                 tweet_id                                         tweet_text  \
0              2327708778  @ajuonline #sarcasm Haal chaal thik thak hai-s...   
1             15150580702           @I_Am_Hollywood - Charlie Haas? #sarcasm   
2             12361902573                   Chetan Bhagat #epicmale #sarcasm   
3       40057904656551936  @Bo_jasim86 LOOL 7atheer affaa 3allaik rgoobti...   
4       39999071498407936  @FENUSF tees2al 3anek el 3afiaa 7abeeb galbe ,...   
...                   ...                                                ...   
201233  38780786895233024  @Javedakhtarjadu salaam alecum sir. aapki izaj...   
201234  38773918928084992  Khud 

In [6]:
df_hinglish_slashed = df_hinglish[['tweet_text']].copy()
df_hinglish_slashed

,tweet_text
0,@ajuonline #sarcasm Haal chaal thik thak hai-s...
1,@I_Am_Hollywood - Charlie Haas? #sarcasm
2,Chetan Bhagat #epicmale #sarcasm
3,@Bo_jasim86 LOOL 7atheer affaa 3allaik rgoobti...
4,"@FENUSF tees2al 3anek el 3afiaa 7abeeb galbe ,..."
...,...
201233,@Javedakhtarjadu salaam alecum sir. aapki izaj...
201234,"Khud Ko Khuda Samjane Wale, Tu Khuda Ka Khouf ..."
201235,"hoti nahi dosti kabhi chehre se, dosti to dil ..."
201236,Jab Jab Dil Dhadkta Hain Tab Tab Tumhari Yaad ...


In [7]:
#finds the duplicate rows in the unlabeled Hinglish data

duplicate_1 = df_hinglish_slashed[df_hinglish_slashed.duplicated(keep='first')]
print(len(duplicate_1))
print(duplicate_1)
#print(type(duplicate_1))

93640
                                               tweet_text
368     Chitti ai hai ai hai chitthi ai hai = I have m...
372     Chitti ai hai ai hai chitthi ai hai = I have m...
393                    Kim Kardashian is virgin. #sarcasm
394                    Kim Kardashian is virgin. #sarcasm
423                    Kim Kardashian is virgin. #sarcasm
...                                                   ...
200973  http://tinyurl.com/32k296l / LOVE QUOTES / Hic...
201054  namskar ek sher aapke kadmo me- "KHAR CHUBHE K...
201171  http://tinyurl.com/4z6sz59 / LOVE QUOTES / Ham...
201196  http://tinyurl.com/5vbq3rh / POETRY QUOTES / K...
201207  http://tinyurl.com/32k296l / LOVE QUOTES / Hic...

[93640 rows x 1 columns]


In [8]:
#removes duplicates

df_hinglish_slashed = df_hinglish_slashed.drop_duplicates(keep='first')
print(len(df_hinglish_slashed))

duplicate_2 = df_hinglish_slashed[df_hinglish_slashed.duplicated(keep='first')]
print(len(duplicate_2))
print(df_hinglish_slashed.columns)

107598
0
Index(['tweet_text'], dtype='object')


In [9]:
df_labeled = pd.read_pickle('dataset/data_soumyadeep.pkl')
df_labeled = df_labeled[['tweet_text', 'is_sarcastic']].copy()
print(len(df_labeled))
print(df_labeled.columns)

94788
Index(['tweet_text', 'is_sarcastic'], dtype='object')


In [10]:
#finds the duplicate rows in the labeled data

duplicate_3 = df_labeled[df_labeled.duplicated(subset=['tweet_text'], keep='first')]
print(len(duplicate_3))
print(duplicate_3)

2895
                                              tweet_text  is_sarcastic
354    RT Apna vyauhar hoga aise jal mein kamal rehat...             0
630    @fittaymunhapka n osky baad amMi ny kha k kuch...             0
686    RT Teen behan ke baad jo aata putra woh trekha...             0
809    Ek aadmi pareshan ho kar Ae bhagwan aisi zinda...             1
844    RT Teen behan ke baad jo aata putra woh trekha...             0
...                                                  ...           ...
94716  RT Teen behan ke baad jo aata putra woh trekha...             0
94721  Roz badh'taa huun jahaa'n se aage // Phir wahi...             0
94747  RT Chhoota vaibhav schooli shiksha shooro ho g...             0
94756                                kya hoga is desh ka             0
94763  @rjauhari lol  unka bhi mast hoga..koi recessi...             0

[2895 rows x 2 columns]


In [11]:
#removes duplicates from labeled data

df_labeled = df_labeled.drop_duplicates(subset=['tweet_text'], keep='first')
print(len(df_labeled))

duplicate_4 = df_labeled[df_labeled.duplicated(keep='first')]
print(len(duplicate_4))
print(df_labeled.columns)

91893
0
Index(['tweet_text', 'is_sarcastic'], dtype='object')


In [12]:
#finds the common rows between unlabeled Hinglish embedding data and labeled data

common_rows = pd.merge(df_hinglish_slashed, df_labeled, how = 'inner', on=['tweet_text'])
print(common_rows)
len(common_rows)
#print(type(common_rows))

                                              tweet_text  is_sarcastic
0      @ajuonline #sarcasm Haal chaal thik thak hai-s...             1
1               @I_Am_Hollywood - Charlie Haas? #sarcasm             1
2                       Chetan Bhagat #epicmale #sarcasm             1
3      @Bo_jasim86 LOOL 7atheer affaa 3allaik rgoobti...             1
4      @FENUSF tees2al 3anek el 3afiaa 7abeeb galbe ,...             1
...                                                  ...           ...
26378  Hurt kerny wala khud bhi hurt ho raha hota hy ...             0
26379  Zindgi me aage badhna hai to 2 baaten yaad rak...             0
26380  @eDeepika  Bus Itna Hi CHahna Mere Baad Merii ...             0
26381  18FEB:Jaruri Nahi Aap Her Galti Khud Ker Ke Si...             0
26382  18FEB:Jaruri Nahi Aap Her Galti Khud Ker Ke Si...             0

[26383 rows x 2 columns]


26383

In [13]:
#removes all the common rows from unlabeled data

common = df_hinglish_slashed['tweet_text'].isin(df_labeled['tweet_text'])
df_hinglish_slashed.drop(df_hinglish_slashed[common].index, inplace=True)
print(len(df_hinglish_slashed))
print(df_hinglish_slashed.columns)

common_rows2 = pd.merge(df_hinglish_slashed, df_labeled, how = 'inner', on=['tweet_text'])
#print(common_rows2)
print(len(common_rows2))

81215
Index(['tweet_text'], dtype='object')
0


/tmp/ipykernel_5751/3615633645.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_hinglish_slashed.drop(df_hinglish_slashed[common].index, inplace=True)


In [14]:
#removes all rows from the Dataframe containing Devanagari script (any non-English word) 

df_hinglish_slashed = df_hinglish_slashed[df_hinglish_slashed['tweet_text'].map(lambda x: x.isascii())]
#filtered_data.iloc[0:5, 1:4]
df_hinglish_slashed.iloc[4, 0]

'3ajeeb make up hayoona #sarcasm'

In [15]:
#removes mentions, '#', emojis, punctuations, special chars, URLs, words containing numbers, spaces from beginning and end of tweets

def data_preprocessing(source):
    
    source = source.str.lower() #lower case chars
    source = source.str.replace("\w*\d\w*","") #removes all words containing numbers
    #source = source.str.replace("#", "")
    #source = source.str.replace("@", "")
    #source = source.str.replace("@[A-Za-z0-9_]+","")
    source = source.str.replace("[@A-Za-z0-9_]*@[@A-Za-z0-9_]*","")
    #source = source.str.replace("#[A-Za-z0-9_]+","")
    source = source.str.replace("[#A-Za-z0-9_]*#[#A-Za-z0-9_]*","")
    source = source.str.replace("\s+|[!$%&()*+, -./:;<=>?\^_`{|}~]\s*"," ")
    source = source.str.replace("http\S+","")
    
    return source

df_hinglish_slashed['tweet_text'] = data_preprocessing(df_hinglish_slashed ['tweet_text'])
df_hinglish_slashed['tweet_text'] = df_hinglish_slashed['tweet_text'].apply(lambda s: emoji.replace_emoji(s,""))
df_hinglish_slashed['tweet_text'] = df_hinglish_slashed['tweet_text'].str.strip()
df_hinglish_slashed[['tweet_text']].head(5)

/tmp/ipykernel_5751/3460830347.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  source = source.str.replace("\w*\d\w*","") #removes all words containing numbers
/tmp/ipykernel_5751/3460830347.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  source = source.str.replace("[@A-Za-z0-9_]*@[@A-Za-z0-9_]*","")
/tmp/ipykernel_5751/3460830347.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  source = source.str.replace("[#A-Za-z0-9_]*#[#A-Za-z0-9_]*","")
/tmp/ipykernel_5751/3460830347.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  source = source.str.replace("\s+|[!$%&()*+, -./:;<=>?\^_`{|}~]\s*"," ")
/tmp/ipykernel_5751/3460830347.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  source = source.str.replace("http\S+","")


,tweet_text
167,dary emratiba
194,yograj singh dhoni advised dd management to r...
200,seedhe log hai ma beta ab crore ka ghotala ho...
309,ab ham khade ho gaye hain ab ham aur bhi dar...
470,make up hayoona


In [16]:
df_hinglish_slashed.iloc[4, 0]

'make up hayoona'

In [17]:
#finds the unique words in the unlabeled data

unique_words = set()
df_hinglish_slashed['tweet_text'].str.lower().str.split().apply(unique_words.update)
unique_words = list(unique_words)
#print(unique_words)
#print('---------------')
print(len(unique_words))

78776


In [18]:
#counts the appearances of the words in the corpus

results = Counter()
df_hinglish_slashed['tweet_text'].str.lower().str.split().apply(results.update)
print(results)
#print('-----------------')
#print(type(results))
#print('-----------------')

Counter({'hai': 37560, 'ki': 21751, 'ko': 16977, 'se': 16590, 'to': 14662, 'ka': 14077, 'ke': 13879, 'nahi': 13253, 'aur': 12559, 'bhi': 12549, 'ho': 12215, 'kabhi': 11666, 'k': 11391, 'bhai': 11049, 'sirf': 9633, 'karne': 9510, 'me': 9495, 'na': 8521, 'hi': 8027, 'phir': 7862, 'hain': 7780, 'http': 7439, 'kar': 6953, 'notebandi': 6554, 'kya': 6517, 'ye': 6041, 'mein': 5821, 'liye': 5553, 'dobara': 5488, 'ne': 5213, 'main': 5080, 'ek': 5034, 'jo': 4968, 'koi': 4787, 'tha': 4764, 'kuch': 4669, 'aap': 4515, 'hum': 4191, 'par': 4187, 'ab': 4027, 'mera': 3800, 'wo': 3710, 'tum': 3565, 'he': 3280, 'h': 3121, 'baat': 3116, 'pe': 3040, 'or': 3011, 'tu': 2971, 'is': 2945, 'raha': 2929, 'aaj': 2902, 'log': 2843, 'toh': 2814, 'nhi': 2772, 'dil': 2755, 'kiya': 2728, 'ji': 2525, 'gaya': 2507, 'hota': 2499, 'yeh': 2482, 'sab': 2462, 'p': 2425, 'kisi': 2411, 'mujhe': 2404, 'rahe': 2367, 'desh': 2347, 't': 2301, 'ly': 2272, 'mai': 2251, 'do': 2235, 'mere': 2216, 'the': 2214, 'modi': 2177, 'e': 2148, 

In [21]:
#finds the rare words in the unlabeled data

rare_words_unlabeled = []
for item in results:
    if results[item]<10:
       #print(item, results[item])
       rare_words_unlabeled.append(item)
        
print(rare_words_unlabeled)  
print('-----------------------')
print(len(rare_words_unlabeled))
print('-----------------------')

with open("rare_words_hing.txt", "w") as file:
    file.write(str(rare_words_unlabeled))

['dary', 'emratiba', 'yograj', 'advised', 'daraayenge', 'hayoona', 'seekhay', 'insey', 'chaata', 'buta', 'vaio', 'mini', 'aanaayi', 'janichirunengil', 'kashtapettu', 'undakki', 'flat', 'okke', 'vangendi', 'vannene', 'bhagyam', 'pennayond', 'athu', 'vendallo', 'bulane', 'patrol', 'casts', 'hypocrisy', 'sarcasm', 'sage', 'smith', 'hence', 'underrated', 'phatake', 'phod', 'aasoo', 'maqsood', 'types', 'gracy', 'dilwado', 'qadhafi', 'hails', 'eu', 'vstz', 'kalavati', 'front', 'surprise', 'austerity', 'ghoos', 'kochi', 'resident', 'nominated', 'admit', 'mehsud', 'hflv', 'rehabilitation', 'lankan', 'tamils', 'bimal', 'celebrates', 'toop', 'basiji', 'deegar', 'nadarad', 'brides', 'virginity', 'tests', 'heat', 'admitted', 'gopalkrishna', 'greets', 'safety', 'welfare', 'rameshwar', 'takes', 'pokhriyal', 'winds', 'greet', 'shivshankar', 'menon', 'koshiyari', 'quits', 'sarvajan', 'bahujan', 'ideological', 'bind', 'congratulated', 'five', 'berths', 'nil', 'retain', 'kannauj', 'buddhadeb', 'tulf', '

In [22]:
with open("rare_words_hing.txt", "r") as file:
    rare_words_unlabeled = eval(file.readline())

#print(rare_words, type(rare_words), type(rare_words[0]))

In [23]:
#--->don't run it now
#removes rare words from the unlabeled data

df_hinglish_slashed['tweet_text'] = df_hinglish_slashed['tweet_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (rare_words_unlabeled)]))
#df_hinglish_slashed[['tweet_text']].head(5)
print(df_hinglish_slashed.head())

                                            tweet_text
167                                                   
194      singh dhoni dd management to release yuvraj p
200  seedhe log hai ma beta ab crore ka ghotala ho ...
309           ab ham khade ho gaye hain ab ham aur bhi
470                                            make up


In [24]:
#--->don't run it now
#saves the preprocessed dataframe to a pickle file

print(df_hinglish_slashed[['tweet_text']].head())
df_hinglish_slashed.to_pickle('preprocessed_hinglish.pkl')

                                            tweet_text
167                                                   
194      singh dhoni dd management to release yuvraj p
200  seedhe log hai ma beta ab crore ka ghotala ho ...
309           ab ham khade ho gaye hain ab ham aur bhi
470                                            make up


In [31]:
#loads the preprocessed dataframe from a pickle file

df_hinglish_slashed = pd.read_pickle('preprocessed_hinglish.pkl')
#print(df[['tweet_text']].head(5))
print(df_hinglish_slashed.head())

                                            tweet_text
167                                                   
194      singh dhoni dd management to release yuvraj p
200  seedhe log hai ma beta ab crore ka ghotala ho ...
309           ab ham khade ho gaye hain ab ham aur bhi
470                                            make up


In [32]:
#finds the unique words in the unlabeled data after rare words removal

unique_words = set()
df_hinglish_slashed['tweet_text'].str.lower().str.split().apply(unique_words.update)
unique_words = list(unique_words)
#print(unique_words)
#print('---------------')
print(len(unique_words))

9156


In [33]:
#generates tokenized data for the preprocessed data

#nltk.download('punkt')
df_hinglish_slashed['tweet_text'] = df_hinglish_slashed['tweet_text'].apply(nltk.word_tokenize)
print(df_hinglish_slashed.head(10))
print(len(df_hinglish_slashed))

                                             tweet_text
167                                                  []
194   [singh, dhoni, dd, management, to, release, yu...
200   [seedhe, log, hai, ma, beta, ab, crore, ka, gh...
309   [ab, ham, khade, ho, gaye, hain, ab, ham, aur,...
470                                          [make, up]
571             [cut, marna, koi, umar, se, nahi, haha]
901   [u, r, aapne, nahi, ki, tarah, pehle, ab, modi...
1073                     [bhai, tu, ek, laptop, le, le]
1273                                             [cash]
1377  [mere, se, koi, aata, hi, nhi, sab, bolte, tu,...
73911


In [34]:
#removes all the rows with empty lists

df_hinglish_slashed=df_hinglish_slashed[df_hinglish_slashed['tweet_text'].map(lambda d: len(d)) > 0]
print(df_hinglish_slashed.head(10))
print(len(df_hinglish_slashed))

                                             tweet_text
194   [singh, dhoni, dd, management, to, release, yu...
200   [seedhe, log, hai, ma, beta, ab, crore, ka, gh...
309   [ab, ham, khade, ho, gaye, hain, ab, ham, aur,...
470                                          [make, up]
571             [cut, marna, koi, umar, se, nahi, haha]
901   [u, r, aapne, nahi, ki, tarah, pehle, ab, modi...
1073                     [bhai, tu, ek, laptop, le, le]
1273                                             [cash]
1377  [mere, se, koi, aata, hi, nhi, sab, bolte, tu,...
1758  [sahi, kaha, ke, ko, le, aana, was, much, much...
73779


In [35]:
model = Word2Vec(df_hinglish_slashed['tweet_text'], vector_size=300, window=10, hs=0, negative = 1)
embedding_size = model.wv.vectors.shape[1]
print("embedding size--->", embedding_size)
vocab = model.wv.index_to_key

embedding size---> 300


In [36]:
#saves the Word2Vec model

model.save("hing_eng_w2v.model")
print("Model saved!!!")
print('-----------------------')
word_vectors = model.wv
print(word_vectors)
word_vectors.save("word2vec_hing_eng.wordvectors")
#print("Word vectors saved !!!")
#X= list(model.wv.vocab)

Model saved!!!
-----------------------


In [37]:
embeddings_index = Word2Vec.load("hing_eng_w2v.model")
print(embeddings_index)

Word2Vec(vocab=9156, vector_size=300, alpha=0.025)


In [38]:
print(df_labeled.head())
print(len(df_labeled))

                                          tweet_text  is_sarcastic
0  LOL !  BC badaa gaandu hai to ! so jaa lodu ! ...             1
1  @OFFICIAL_FC_PAK apni sakh khud sambhalen aur ...             0
2  Ek aadmi pareshan ho kar Ae bhagwan aisi zinda...             1
3  Lol RT @ThePrachi: @shrikhande Who can sing a ...             1
4  @idrinkblueblood tu ol's kar rahi ho kya? Kya ...             0
91893


In [39]:
#removes mentions, '#', emojis, punctuations, special chars, URLs, words containing numbers, spaces from beginning and end of tweets

def data_preprocessing(source):
    
    source = source.str.lower() #lower case chars
    source = source.str.replace("\w*\d\w*","") #removes all words containing numbers
    #source = source.str.replace("#", "")
    #source = source.str.replace("@", "")
    #source = source.str.replace("@[A-Za-z0-9_]+","")
    source = source.str.replace("[@A-Za-z0-9_]*@[@A-Za-z0-9_]*","")
    #source = source.str.replace("#[A-Za-z0-9_]+","")
    source = source.str.replace("[#A-Za-z0-9_]*#[#A-Za-z0-9_]*","")
    source = source.str.replace("\s+|[!$%&()*+, -./:;<=>?\^_`{|}~]\s*"," ")
    source = source.str.replace("http\S+","")
    
    return source

df_labeled['tweet_text'] = data_preprocessing(df_labeled['tweet_text'])
df_labeled['tweet_text'] = df_labeled['tweet_text'].apply(lambda s: emoji.replace_emoji(s,""))
df_labeled['tweet_text'] = df_labeled['tweet_text'].str.strip()
df_labeled[['tweet_text']].head()

/tmp/ipykernel_5751/3018622983.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  source = source.str.replace("\w*\d\w*","") #removes all words containing numbers
/tmp/ipykernel_5751/3018622983.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  source = source.str.replace("[@A-Za-z0-9_]*@[@A-Za-z0-9_]*","")
/tmp/ipykernel_5751/3018622983.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  source = source.str.replace("[#A-Za-z0-9_]*#[#A-Za-z0-9_]*","")
/tmp/ipykernel_5751/3018622983.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  source = source.str.replace("\s+|[!$%&()*+, -./:;<=>?\^_`{|}~]\s*"," ")
/tmp/ipykernel_5751/3018622983.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  source = source.str.replace("http\S+","")


,tweet_text
0,lol bc badaa gaandu hai to so jaa lodu tere...
1,apni sakh khud sambhalen aur kisi ka saath na ...
2,ek aadmi pareshan ho kar ae bhagwan aisi zinda...
3,lol rt who can sing a group song alone raavan...
4,tu ol s kar rahi ho kya kya hoga tere kaam sir...


In [40]:
df_labeled.head()

,tweet_text,is_sarcastic
0,lol bc badaa gaandu hai to so jaa lodu tere...,1
1,apni sakh khud sambhalen aur kisi ka saath na ...,0
2,ek aadmi pareshan ho kar ae bhagwan aisi zinda...,1
3,lol rt who can sing a group song alone raavan...,1
4,tu ol s kar rahi ho kya kya hoga tere kaam sir...,0


In [41]:
#finds the unique words in the labeled data

unique_words = set()
df_labeled['tweet_text'].str.lower().str.split().apply(unique_words.update)
unique_words = list(unique_words)
#print(unique_words)
#print('---------------')
print(len(unique_words))

93958


In [42]:
#counts the appearances of the words in the labeled data

results = Counter()
df_labeled['tweet_text'].str.lower().str.split().apply(results.update)
print(results)
#print('-----------------')
#print(type(results))
#print('-----------------')

Counter({'hai': 51160, 'ki': 21224, 'ke': 19471, 'se': 18439, 'ko': 18377, 'to': 17606, 'bhi': 16969, 'ka': 15888, 'nahi': 15425, 'ho': 14862, 'k': 14337, 'hoga': 12832, 'dil': 12760, 'aur': 11643, 'aage': 11483, 'baad': 11214, 'me': 11156, 'hi': 10120, 'na': 9972, 'hain': 9662, 'kya': 9596, 'mazak': 9185, 'kar': 9114, 'lol': 8834, 'ye': 8071, 'koi': 6862, 'mein': 6813, 'kuch': 6542, 'ne': 6458, 'tha': 6395, 'ek': 6339, 'jo': 5997, 'main': 5704, 'aap': 5463, 'bhai': 5382, 'ab': 5193, 'serious': 5193, 'raha': 4966, 'wo': 4696, 'toh': 4684, 'hum': 4598, 'liye': 4391, 'tum': 4304, 'he': 4165, 'par': 4148, 'baat': 4139, 'tu': 4071, 'is': 4069, 'aaj': 4048, 'pe': 4026, 't': 4003, 'h': 3795, 'hota': 3739, 'mere': 3680, 'p': 3633, 'co': 3570, 'nhi': 3528, 'kisi': 3516, 'or': 3499, 'kiya': 3492, 'log': 3454, 'mujhe': 3444, 'http': 3417, 'kabhi': 3328, 'sab': 3240, 'mai': 3234, 'rahe': 3206, 'ha': 3167, 'gaya': 3154, 'yeh': 2979, 'satire': 2903, 'jab': 2864, 'funny': 2766, 'e': 2716, 'ji': 2639

In [43]:
#finds the rare words in the labeled data

rare_words_labeled = []
for item in results:
    if results[item]<10:
       #print(item, results[item])
       rare_words_labeled.append(item)
        
print(rare_words_labeled)  
print('-----------------------')
print(len(rare_words_labeled))
print('-----------------------')

with open("rare_words_labeled.txt", "w") as file:
    file.write(str(rare_words_labeled))

['lodu', 'sakh', 'sambhalen', 'rhne', 'yamdut', 'marate', 'rehete', 'onkine', 'bhokon', 'tauko', 'dukhyo', 'vanera', 'manchhe', 'sadhai', 'dukhirachha', 'vaira', 'saansain', 'adhoori', 'kamiya', 'sunnegey', 'dogey', 'dengey', 'ramrajey', 'osky', 'bohaat', 'kunda', 'liakin', 'aundhi', 'jeh', 'bimmar', 'pawa', 'puchda', 'ethe', 'kudiya', 'hundeh', 'rasmein', 'poorani', 'gathring', 'deten', 'qayanat', 'atharv', 'dogu', 'iwas', 'inoe', 'chood', 'badme', 'baitho', 'doobta', 'pedo', 'wissaley', 'shairr', 'hyn', 'barg', 'bijnesman', 'badlne', 'burkha', 'hahahakar', 'thuuuu', 'vicks', 'baana', 'panama', 'velapanti', 'isee', 'kashmakash', 'lambai', 'chorai', 'relatable', 'chanta', 'chadd', 'gavaani', 'phail', 'chhutkara', 'tweetoo', 'humale', 'manchild', 'crybaby', 'rehab', 'returned', 'aggre', 'ive', 'damaadji', 'khudke', 'shookla', 'physical', 'barobar', 'boltes', 'evda', 'todte', 'hawaldar', 'baataho', 'tuja', 'atiha', 'asliya', 'pasat', 'subtweet', 'chiriya', 'modu', 'hastag', 'marane', 'ph

In [44]:
with open("rare_words_labeled.txt", "r") as file:
    rare_words_labeled = eval(file.readline())

In [45]:
#--->don't run it now
#removes rare words from the labeled data

df_labeled['tweet_text'] = df_labeled['tweet_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (rare_words_labeled)]))
print(df_labeled.head())

                                          tweet_text  is_sarcastic
0  lol bc badaa gaandu hai to so jaa tere se kuch...             1
1  apni khud aur kisi ka saath na dein yehi log b...             0
2  ek aadmi pareshan ho kar ae bhagwan aisi zinda...             1
3  lol rt who can sing a group song alone raavan ...             1
4  tu ol s kar rahi ho kya kya hoga tere kaam sir...             0


In [46]:
#--->don't run it now
#saves the preprocessed dataframe to a pickle file

print(df_labeled.head())
df_labeled.to_pickle('preprocessed_labeled_hing.pkl')

                                          tweet_text  is_sarcastic
0  lol bc badaa gaandu hai to so jaa tere se kuch...             1
1  apni khud aur kisi ka saath na dein yehi log b...             0
2  ek aadmi pareshan ho kar ae bhagwan aisi zinda...             1
3  lol rt who can sing a group song alone raavan ...             1
4  tu ol s kar rahi ho kya kya hoga tere kaam sir...             0


In [47]:
#loads the preprocessed dataframe from a pickle file

df_labeled = pd.read_pickle('preprocessed_labeled_hing.pkl')
print(df_labeled.head())
print(len(df_labeled))

                                          tweet_text  is_sarcastic
0  lol bc badaa gaandu hai to so jaa tere se kuch...             1
1  apni khud aur kisi ka saath na dein yehi log b...             0
2  ek aadmi pareshan ho kar ae bhagwan aisi zinda...             1
3  lol rt who can sing a group song alone raavan ...             1
4  tu ol s kar rahi ho kya kya hoga tere kaam sir...             0
91893


In [48]:
#finds the unique words in the labeled data after rare words removal

unique_words = set()
df_labeled['tweet_text'].str.lower().str.split().apply(unique_words.update)
unique_words = list(unique_words)
#print(unique_words)
#print('---------------')
print(len(unique_words))

10014


In [49]:
#generates tokenized data for the preprocessed data

#nltk.download('punkt')
df_labeled['tweet_text'] = df_labeled['tweet_text'].apply(nltk.word_tokenize)
df_labeled.head()

,tweet_text,is_sarcastic
0,"[lol, bc, badaa, gaandu, hai, to, so, jaa, ter...",1
1,"[apni, khud, aur, kisi, ka, saath, na, dein, y...",0
2,"[ek, aadmi, pareshan, ho, kar, ae, bhagwan, ai...",1
3,"[lol, rt, who, can, sing, a, group, song, alon...",1
4,"[tu, ol, s, kar, rahi, ho, kya, kya, hoga, ter...",0


In [50]:
#removes all the rows with empty lists

print(len(df_labeled))
df_labeled = df_labeled[df_labeled['tweet_text'].map(lambda d: len(d)) > 0]
print(df_labeled.head(10))
print(len(df_labeled))

91893
                                          tweet_text  is_sarcastic
0  [lol, bc, badaa, gaandu, hai, to, so, jaa, ter...             1
1  [apni, khud, aur, kisi, ka, saath, na, dein, y...             0
2  [ek, aadmi, pareshan, ho, kar, ae, bhagwan, ai...             1
3  [lol, rt, who, can, sing, a, group, song, alon...             1
4  [tu, ol, s, kar, rahi, ho, kya, kya, hoga, ter...             0
5      [yaar, ye, log, tu, serious, ho, gaye, t, co]             0
6  [abey, tumhare, paas, kaam, nahi, hai, jo, twi...             0
7  [ani, jaane, ta, ho, ni, o, hawa, serious, kei...             0
8  [rt, agar, tumne, rt, nahi, kiya, to, mujhse, ...             0
9  [dil, ki, dhadkan, sooni, sooni, bhi, adha, ad...             0
91557


In [51]:
df_labeled.head()

,tweet_text,is_sarcastic
0,"[lol, bc, badaa, gaandu, hai, to, so, jaa, ter...",1
1,"[apni, khud, aur, kisi, ka, saath, na, dein, y...",0
2,"[ek, aadmi, pareshan, ho, kar, ae, bhagwan, ai...",1
3,"[lol, rt, who, can, sing, a, group, song, alon...",1
4,"[tu, ol, s, kar, rahi, ho, kya, kya, hoga, ter...",0


In [53]:
df_hinglish_slashed.head()

,tweet_text
194,"[singh, dhoni, dd, management, to, release, yu..."
200,"[seedhe, log, hai, ma, beta, ab, crore, ka, gh..."
309,"[ab, ham, khade, ho, gaye, hain, ab, ham, aur,..."
470,"[make, up]"
571,"[cut, marna, koi, umar, se, nahi, haha]"


In [54]:
#train test split of labeled data

check_nan = df_labeled['is_sarcastic'].isnull().values.any()
print(check_nan)
check_na = df_labeled['is_sarcastic'].isna().values.any()
print(check_na)
check_nan_ = df_labeled['tweet_text'].isnull().values.any()
print(check_nan_)
check_na_ = df_labeled['tweet_text'].isna().values.any()
print(check_na_)

x_train,x_test,y_train,y_test=train_test_split(df_labeled['tweet_text'], df_labeled['is_sarcastic'], test_size=0.30, random_state=42)
#x_train,x_test,y_train,y_test=train_test_split(padded_tweets, sarcasm_labels, test_size=0.20, random_state=42)
print("x_train_shape--->", x_train.shape)
print("y_train_shape--->", y_train.shape)
print("x_test_shape--->", x_test.shape)
print("y_test_shape--->", y_test.shape)
print(type(x_train))
print("contents of first label--->", y_train[0])

False
False
False
False
x_train_shape---> (64089,)
y_train_shape---> (64089,)
x_test_shape---> (27468,)
y_test_shape---> (27468,)
<class 'pandas.core.series.Series'>
contents of first label---> 1


In [55]:
print(x_train)

42486    [kya, baat, i, ll, say, this, while, the, awar...
81430    [koi, baat, nahi, arjun, yeh, prem, rog, hai, ...
16313    [hum, uss, desh, ke, wasi, hai, jahan, roz, hi...
25490    [indian, parents, ko, badi, badi, baaton, se, ...
69537    [koun, kehta, h, ki, yeh, paise, lete, hain, y...
                               ...                        
6337     [filhal, to, bechare, ko, se, baher, nikal, dy...
56392                                  [boat, http, t, co]
79310    [galti, se, mistake, ho, gaye, lol, ab, kya, k...
868      [to, kese, k, jao, nai, bat, nai, udasi, aage,...
16071                                      [lol, jai, jai]
Name: tweet_text, Length: 64089, dtype: object


In [56]:
print(y_train.value_counts())
print('--------------------------')
print(y_test.value_counts())

0    32461
1    31628
Name: is_sarcastic, dtype: int64
--------------------------
0    13943
1    13525
Name: is_sarcastic, dtype: int64


In [57]:
length = []
for row in x_train:
    #print(row)
    #print(len(row))
    #count = count+1
    #if len(row)>maxi:
    #print(len(row))
    length.append(len(row))
        
#print(length)
print("Max length--->", max(length))
print("Mean length--->", np.mean(length))
print("Standard Deviation of length--->", np.std(length))

#maxi = max(length)
maxi = np.round(np.mean(length) + np.std(length)).astype(int)
print("Maximum length of the sequence--->", maxi)

Max length---> 79
Mean length---> 14.970790619295043
Standard Deviation of length---> 8.82197247942889
Maximum length of the sequence---> 24


In [58]:
total_docs=[]
total_docs.extend(x_train)
total_docs.extend(x_test)
print(total_docs[0:5])

[['kya', 'baat', 'i', 'll', 'say', 'this', 'while', 'the', 'award', 'jab', 'sharafat', 'ke', 'kapde', 'hai', 'tab', 'sabse', 'jyaada', 'maja'], ['koi', 'baat', 'nahi', 'arjun', 'yeh', 'prem', 'rog', 'hai', 'shuru', 'me', 'dukh', 'deta', 'hai', 'aur', 'baad', 'me', 'malik', 'baad', 'me', 'bahut', 'dukh', 'deta', 'hai', 'd'], ['hum', 'uss', 'desh', 'ke', 'wasi', 'hai', 'jahan', 'roz', 'hi', 'panga', 'rehta', 'hai', 'jahan', 'roz', 'roz', 'me', 'ka', 'panga', 'rehta', 'hai', 'iss', 'haal', 'me', 'bhi', 'iss', 'dil', 'me', 'bas', 'ek', 'tiranga', 'rehta', 'hai', 'believe', 'me', 'or', 'die', 'this', 'scene', 'jai', 'hind', 'of', 'india', 't', 'co'], ['indian', 'parents', 'ko', 'badi', 'badi', 'baaton', 'se', 'nahi', 'pe', 'lage', 'se', 'pata', 'chalta', 'hai', 'bache', 'bade', 'ho', 'gaye', 'hai'], ['koun', 'kehta', 'h', 'ki', 'yeh', 'paise', 'lete', 'hain', 'yeh', 'toh', 'bass', 'desh', 'ki', 'karne', 'ke', 'liye', 'bina', 'paise', 'ke', 'kaam', 'kar', 'rhe', 'hain', 'galat', 'toh', 'this

In [59]:
print("tokenizing input data...")
MAX_NB_WORDS = 200000
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, lower=True, char_level=False)
tokenizer.fit_on_texts(total_docs)
#tokenizer.fit_on_texts(x_train)
word_seq_train = tokenizer.texts_to_sequences(x_train)
word_seq_test = tokenizer.texts_to_sequences(x_test)
word_index = tokenizer.word_index

tokenizing input data...


In [60]:
print(len(word_seq_train))
print(word_seq_train[0])
print(len(word_seq_test))
print(word_seq_test[0])
print("dictionary size: ", len(word_index))

64089
[21, 46, 109, 3592, 183, 334, 4561, 102, 2103, 72, 6955, 3, 4209, 1, 253, 394, 3425, 2024]
27468
[94, 604, 131, 21, 12, 63, 51, 56]
dictionary size:  10013


In [61]:
#pads to the length of the longest sequence

word_seq_train = pad_sequences(word_seq_train, maxlen=maxi, padding='post')
word_seq_test = pad_sequences(word_seq_test, maxlen=maxi, padding='post')
print(word_seq_train[0])
print(word_seq_train.shape)
print(word_seq_test.shape)

[  21   46  109 3592  183  334 4561  102 2103   72 6955    3 4209    1
  253  394 3425 2024    0    0    0    0    0    0]
(64089, 24)
(27468, 24)


In [62]:
#builds the embedding matrix

print('preparing embedding matrix...')
embed_dim = embedding_size
words_not_found = []
nb_words = min(MAX_NB_WORDS, len(word_index))
embedding_matrix = np.random.rand(nb_words+1, embed_dim)

for word, i in word_index.items():
    if i >= nb_words:
        continue
    #print(word)

    if embeddings_index.wv.__contains__(word): 

        embedding_vector = embeddings_index.wv[word]
    
        embedding_matrix[i] = embedding_vector
    else:
        words_not_found.append(word)
#print(embedding_matrix)
print('number of null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))
print(embedding_matrix.shape)
print(len(embedding_matrix[34]))

preparing embedding matrix...
number of null word embeddings: 0
(10014, 300)
300


In [64]:
print(words_not_found)
print(len(words_not_found))

['serious', 'satire', 'mock', 'taunt', 'joking', 'mocks', 'cli', 'badho', 'drill', 'jhakaas', 'sarcasm', 'ura', 'shairy', 'udaya', 'bado', 'mee', 'udate', 'shuruvaat', 'uraya', 'badhne', 'cheer', 'doraniya', 'teetar', 'kaghaz', 'sarafa', 'yuu', 'association', 'mahira', 'aneh', 'udana', 'taweel', 'maghrib', 'badne', 'wya', 'mukhtasir', 'udane', 'samjhaya', 'faraaz', 'muddaton', 'badhe', 'jeejee', 'rakt', 'meme', 'ghanty', 'badhane', 'mehkta', 'biggboss', 'hola', 'chaaon', 'sheesh', 'kilo', 'marsha', 'waa', 'mazahiya', 'taise', 'yuup', 'jeans', 'layegi', 'uratay', 'stadium', 'urana', 'aali', 'humour', 'bakri', 'mobi', 'maarta', 'barhne', 'dha', 'fandom', 'udao', 'udata', 'bhagvan', 'ew', 'priyank', 'bhaage', 'sidhe', 'sitaaron', 'raahi', 'dukhta', 'badhta', 'tazeem', 'em', 'every', 'nau', 'sitaro', 'doosray', 'majaak', 'khushki', 'jalaya', 'rakht', 'saur', 'dukhaya', 'reet', 'shaiqeen', 'hamrah', 'hass', 'humor', 'kabhe', 'bebas', 'crime', 'shamma', 'piyari', 'badhana', 'ladko', 'arshi',

In [65]:
#words not found in the embeddings

print("sample words not found: ", np.random.choice(words_not_found, 10))

sample words not found:  ['mukhtalif' 'leka' 'khabri' 'gulon' 'dukhao' 'dugi' 'oru' 'fekne'
 'jalebi' 'kila']


In [66]:
#define callbacks
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=4, verbose=1)
callbacks_list = [early_stopping]

In [67]:
#Attention BiLSTM

from keras import backend as K
#from keras.engine.topology import Layer
from tensorflow.keras.layers import Layer
from keras import initializers, regularizers, constraints


class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight( shape=(input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]
        #print(self.name)
        if self.bias:
            self.b = self.add_weight(shape=(input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim


In [68]:
lstm_out1 = 150
#bilstm_out1 = 96
#lstm_out2 = 32
#dropout = 0.2
#recurrent_dropout = 0.2

inp = Input(shape=(maxi, ))
x = Embedding(nb_words+1, embed_dim, weights=[embedding_matrix], trainable=False)(inp)
x = Bidirectional(LSTM(lstm_out1, return_sequences=True, dropout=0.20, recurrent_dropout=0.20))(x)
x = Attention(maxi)(x)
x = Dense(256, activation="relu")(x)
#x = Dropout(0.25)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)

adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
model.summary()

2023-04-26 15:46:18.638148: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-04-26 15:46:18.638193: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-04-26 15:46:18.655675: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (user-PC): /proc/driver/nvidia/version does not exist
2023-04-26 15:46:18.679849: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 24)]              0         
                                                                 
 embedding (Embedding)       (None, 24, 300)           3004200   
                                                                 
 bidirectional (Bidirectiona  (None, 24, 300)          541200    
 l)                                                              
                                                                 
 attention (Attention)       (None, 300)               324       
                                                                 
 dense (Dense)               (None, 256)               77056     
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                             

/home/user/anaconda3/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [69]:
#trains attention BiLSTM model 

model.fit(word_seq_train, y_train, batch_size=64, epochs=20, validation_split=0.1, shuffle=True, callbacks = callbacks_list)
#model.save('att_bilstm.h5') 

Epoch 1/20
902/902 [==============================] - 354s 381ms/step - loss: 0.3581 - accuracy: 0.8402 - val_loss: 0.1889 - val_accuracy: 0.9459
Epoch 2/20
902/902 [==============================] - 346s 384ms/step - loss: 0.1681 - accuracy: 0.9535 - val_loss: 0.1647 - val_accuracy: 0.9537
Epoch 3/20
902/902 [==============================] - 350s 389ms/step - loss: 0.1514 - accuracy: 0.9567 - val_loss: 0.1510 - val_accuracy: 0.9576
Epoch 4/20
902/902 [==============================] - 348s 386ms/step - loss: 0.1439 - accuracy: 0.9579 - val_loss: 0.1511 - val_accuracy: 0.9587
Epoch 5/20
902/902 [==============================] - 260s 288ms/step - loss: 0.1372 - accuracy: 0.9585 - val_loss: 0.1471 - val_accuracy: 0.9591
Epoch 6/20
902/902 [==============================] - 260s 288ms/step - loss: 0.1308 - accuracy: 0.9598 - val_loss: 0.1456 - val_accuracy: 0.9594
Epoch 7/20
902/902 [==============================] - 272s 301ms/step - loss: 0.1240 - accuracy: 0.9610 - val_loss: 0.1481 -

In [70]:
#model = load_model('att_bilstm.h5')
y_pred_proba = model.predict(word_seq_test)
#print(y_pred_proba)
#print('---------------')

y_pred_class = np.round(y_pred_proba)
y_pred_class = y_pred_class.squeeze() #makes y_pred_class 1 dim
'''print(y_pred_class)
print(type(y_pred_class))
print(len(y_pred_class))
print(len(y_test))
print('---------------')'''

acc = accuracy_score(y_test, y_pred_class) * 100
print("Accuracy is--->", acc)

859/859 [==============================] - 46s 52ms/step
Accuracy is---> 95.67860783457114


In [71]:
#this one--->LSTM

lstm_out1 = 150
'''bilstm_out1 = 96
lstm_out2 = 32
dropout = 0.2
recurrent_dropout = 0.2'''

model = Sequential()
model.add(Embedding(nb_words+1, embed_dim, weights=[embedding_matrix], input_length=maxi, trainable=False))
model.add(LSTM(lstm_out1, dropout=0.2, recurrent_dropout=0.2))

model.add(Dense(64, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(loss='binary_crossentropy',
                  optimizer=adam,
                  metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 24, 300)           3004200   
                                                                 
 lstm_1 (LSTM)               (None, 150)               270600    
                                                                 
 dense_2 (Dense)             (None, 64)                9664      
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 3,284,529
Trainable params: 280,329
Non-trainable params: 3,004,200
_________________________________________________________________


/home/user/anaconda3/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [72]:
#trains the LSTM model

model.fit(word_seq_train, y_train, batch_size=64, epochs=20, validation_split=0.1, shuffle=True, callbacks = callbacks_list)
model.save('lstm.h5') 

Epoch 1/20
902/902 [==============================] - 128s 138ms/step - loss: 0.4344 - accuracy: 0.7924 - val_loss: 0.2890 - val_accuracy: 0.8827
Epoch 2/20
902/902 [==============================] - 124s 137ms/step - loss: 0.2385 - accuracy: 0.9175 - val_loss: 0.1803 - val_accuracy: 0.9462
Epoch 3/20
902/902 [==============================] - 124s 137ms/step - loss: 0.1705 - accuracy: 0.9487 - val_loss: 0.1591 - val_accuracy: 0.9549
Epoch 4/20
902/902 [==============================] - 124s 138ms/step - loss: 0.1576 - accuracy: 0.9529 - val_loss: 0.1513 - val_accuracy: 0.9549
Epoch 5/20
902/902 [==============================] - 126s 140ms/step - loss: 0.1484 - accuracy: 0.9556 - val_loss: 0.1489 - val_accuracy: 0.9582
Epoch 6/20
902/902 [==============================] - 119s 132ms/step - loss: 0.1418 - accuracy: 0.9570 - val_loss: 0.1499 - val_accuracy: 0.9574
Epoch 7/20
902/902 [==============================] - 119s 132ms/step - loss: 0.1370 - accuracy: 0.9583 - val_loss: 0.1495 -

In [73]:
#model = load_model('att_bilstm.h5')
y_pred_proba = model.predict(word_seq_test)
#print(y_pred_proba)
#print('---------------')

y_pred_class = np.round(y_pred_proba)
y_pred_class = y_pred_class.squeeze() #makes y_pred_class 1 dim
'''print(y_pred_class)
print(type(y_pred_class))
print(len(y_pred_class))
print(len(y_test))
print('---------------')'''

acc = accuracy_score(y_test, y_pred_class) * 100
print("Accuracy is--->", acc)

859/859 [==============================] - 24s 28ms/step
Accuracy is---> 95.49657783602737


In [74]:
# BiLSTM

lstm_out1 = 150
bilstm_out1 = 96
lstm_out2 = 32
dropout = 0.2
recurrent_dropout = 0.2

# main model
inputt = Input(shape=(maxi,))
model = Embedding(nb_words+1, embed_dim, weights=[embedding_matrix],input_length=maxi,trainable = False)(inputt)
model =  Bidirectional (LSTM (lstm_out1, return_sequences=True, dropout=0.2, recurrent_dropout=0.2), merge_mode='concat')(model)
model = TimeDistributed(Dense(10,activation='relu'))(model)
model = Flatten()(model)
model = Dense(64,activation='relu')(model)
output = Dense(1,activation='sigmoid')(model)
model = Model(inputt,output)
adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 24)]              0         
                                                                 
 embedding_2 (Embedding)     (None, 24, 300)           3004200   
                                                                 
 bidirectional_1 (Bidirectio  (None, 24, 300)          541200    
 nal)                                                            
                                                                 
 time_distributed (TimeDistr  (None, 24, 10)           3010      
 ibuted)                                                         
                                                                 
 flatten (Flatten)           (None, 240)               0         
                                                                 
 dense_5 (Dense)             (None, 64)                1542

/home/user/anaconda3/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [75]:
model.fit(word_seq_train, y_train, batch_size=64, epochs=20, validation_split=0.1, shuffle=True, callbacks = callbacks_list)
model.save('bilstm.h5') 

Epoch 1/20
902/902 [==============================] - 291s 314ms/step - loss: 0.4033 - accuracy: 0.8123 - val_loss: 0.2678 - val_accuracy: 0.9014
Epoch 2/20
902/902 [==============================] - 330s 366ms/step - loss: 0.1896 - accuracy: 0.9428 - val_loss: 0.1569 - val_accuracy: 0.9541
Epoch 3/20
902/902 [==============================] - 359s 398ms/step - loss: 0.1505 - accuracy: 0.9565 - val_loss: 0.1458 - val_accuracy: 0.9576
Epoch 4/20
902/902 [==============================] - 344s 381ms/step - loss: 0.1412 - accuracy: 0.9573 - val_loss: 0.1482 - val_accuracy: 0.9585
Epoch 5/20
902/902 [==============================] - 334s 370ms/step - loss: 0.1338 - accuracy: 0.9591 - val_loss: 0.1460 - val_accuracy: 0.9560
Epoch 6/20
902/902 [==============================] - 282s 312ms/step - loss: 0.1255 - accuracy: 0.9601 - val_loss: 0.1397 - val_accuracy: 0.9583
Epoch 7/20
902/902 [==============================] - 273s 302ms/step - loss: 0.1173 - accuracy: 0.9612 - val_loss: 0.1449 -

In [76]:
#model = load_model('att_bilstm.h5')
y_pred_proba = model.predict(word_seq_test)
#print(y_pred_proba)
#print('---------------')

y_pred_class = np.round(y_pred_proba)
y_pred_class = y_pred_class.squeeze() #makes y_pred_class 1 dim
'''print(y_pred_class)
print(type(y_pred_class))
print(len(y_pred_class))
print(len(y_test))
print('---------------')'''

acc = accuracy_score(y_test, y_pred_class) * 100
print("Accuracy is--->", acc)

859/859 [==============================] - 91s 105ms/step
Accuracy is---> 95.4638124362895


In [77]:
#CNN1

#model parameters

num_kernels=200
stride=1
embed_dim = 300
weight_decay = 1e-4

#parallel layers 
kernel_size=7
kernel_size_p1=3
kernel_size_p2=6
kernel_size_p3=9
kernel_size_p4=12

#CNN architecture
print("training CNN ...")
model = Sequential()
model.add(Embedding(nb_words+1, embed_dim, weights=[embedding_matrix], input_length=maxi, trainable=False))
model.add(Conv1D(num_kernels, kernel_size, activation='relu', padding='same'))
model.add(MaxPooling1D(2))
model.add(Conv1D(num_kernels, kernel_size, activation='relu', padding='same'))
#model.add(GlobalMaxPooling1D())
model.add(MaxPooling1D(2))
#model.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
#model.add(Dense(num_classes, activation='sigmoid'))  #multi-label (k-hot encoding)
model.add(Dense(1, activation='sigmoid')) 

adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

training CNN ...
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 24, 300)           3004200   
                                                                 
 conv1d (Conv1D)             (None, 24, 200)           420200    
                                                                 
 max_pooling1d (MaxPooling1D  (None, 12, 200)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 12, 200)           280200    
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 6, 200)           0         
 1D)                                                             
                                                                 
 global_max_pooling1d (Globa  (None, 

/home/user/anaconda3/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [78]:
model.fit(word_seq_train, y_train, batch_size=64, epochs=20, validation_split=0.1, shuffle=True, callbacks = callbacks_list)
model.save('cnn1.h5') 

Epoch 1/20
902/902 [==============================] - 102s 111ms/step - loss: 0.4499 - accuracy: 0.7829 - val_loss: 0.3370 - val_accuracy: 0.8563
Epoch 2/20
902/902 [==============================] - 92s 102ms/step - loss: 0.3242 - accuracy: 0.8651 - val_loss: 0.3123 - val_accuracy: 0.8635
Epoch 3/20
902/902 [==============================] - 128s 141ms/step - loss: 0.2979 - accuracy: 0.8785 - val_loss: 0.2898 - val_accuracy: 0.8816
Epoch 4/20
902/902 [==============================] - 118s 131ms/step - loss: 0.2778 - accuracy: 0.8875 - val_loss: 0.2964 - val_accuracy: 0.8769
Epoch 5/20
902/902 [==============================] - 97s 107ms/step - loss: 0.2627 - accuracy: 0.8955 - val_loss: 0.2923 - val_accuracy: 0.8834
Epoch 6/20
902/902 [==============================] - 104s 115ms/step - loss: 0.2447 - accuracy: 0.9051 - val_loss: 0.2929 - val_accuracy: 0.8841
Epoch 7/20
902/902 [==============================] - 123s 137ms/step - loss: 0.2261 - accuracy: 0.9130 - val_loss: 0.2863 - v

In [79]:
#model = load_model('att_bilstm.h5')
y_pred_proba = model.predict(word_seq_test)
#print(y_pred_proba)
#print('---------------')

y_pred_class = np.round(y_pred_proba)
y_pred_class = y_pred_class.squeeze() #makes y_pred_class 1 dim
'''print(y_pred_class)
print(type(y_pred_class))
print(len(y_pred_class))
print(len(y_test))
print('---------------')'''

acc = accuracy_score(y_test, y_pred_class) * 100
print("Accuracy is--->", acc)

859/859 [==============================] - 15s 17ms/step
Accuracy is---> 88.17533129459734


In [80]:
#CNN2

#num_classes = len(label_names)
num_classes = 2

inputt = Input(shape=(maxi,))
model = Embedding(nb_words+1, embed_dim, weights=[embedding_matrix],input_length=maxi,trainable = False)(inputt)
tower_1 = Conv1D(num_kernels, kernel_size_p1, padding='same', activation='relu', strides=stride)(model)
tower_2 = Conv1D(num_kernels, kernel_size_p2, padding='same', activation='relu', strides=stride)(model)
tower_3 = Conv1D(num_kernels, kernel_size_p3, padding='same', activation='relu', strides=stride)(model)
tower_4 = Conv1D(num_kernels, kernel_size_p4, padding='same', activation='relu', strides=stride)(model)
model = keras.layers.concatenate([tower_1, tower_2, tower_3, tower_4], axis=1)
model = GlobalMaxPooling1D()(model)
model = Dropout(0.5)(model)
#model = Flatten()(model)
model = Dense(128, activation='relu')(model)
model = Dense(64, activation='relu')(model)
#output = Dense(num_classes, activation='sigmoid')(model)
output = Dense(1, activation='sigmoid')(model)
model = Model(inputt, output)

#adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 24)]         0           []                               
                                                                                                  
 embedding_4 (Embedding)        (None, 24, 300)      3004200     ['input_3[0][0]']                
                                                                                                  
 conv1d_2 (Conv1D)              (None, 24, 200)      180200      ['embedding_4[0][0]']            
                                                                                                  
 conv1d_3 (Conv1D)              (None, 24, 200)      360200      ['embedding_4[0][0]']            
                                                                                            

In [81]:
model.fit(word_seq_train, y_train, batch_size=64, epochs=20, validation_split=0.1, shuffle=True, callbacks = callbacks_list)
model.save('cnn2.h5') 

Epoch 1/20
902/902 [==============================] - 198s 218ms/step - loss: 0.5025 - accuracy: 0.7484 - val_loss: 0.3483 - val_accuracy: 0.8508
Epoch 2/20
902/902 [==============================] - 246s 273ms/step - loss: 0.3676 - accuracy: 0.8419 - val_loss: 0.3174 - val_accuracy: 0.8638
Epoch 3/20
902/902 [==============================] - 211s 234ms/step - loss: 0.3384 - accuracy: 0.8565 - val_loss: 0.3081 - val_accuracy: 0.8716
Epoch 4/20
902/902 [==============================] - 219s 243ms/step - loss: 0.3194 - accuracy: 0.8658 - val_loss: 0.3012 - val_accuracy: 0.8719
Epoch 5/20
902/902 [==============================] - 201s 223ms/step - loss: 0.3027 - accuracy: 0.8756 - val_loss: 0.2835 - val_accuracy: 0.8817
Epoch 6/20
902/902 [==============================] - 206s 228ms/step - loss: 0.2848 - accuracy: 0.8876 - val_loss: 0.2596 - val_accuracy: 0.9006
Epoch 7/20
902/902 [==============================] - 235s 261ms/step - loss: 0.2633 - accuracy: 0.8992 - val_loss: 0.2448 -

In [82]:
#model = load_model('att_bilstm.h5')
y_pred_proba = model.predict(word_seq_test)
#print(y_pred_proba)
#print('---------------')

y_pred_class = np.round(y_pred_proba)
y_pred_class = y_pred_class.squeeze() #makes y_pred_class 1 dim
'''print(y_pred_class)
print(type(y_pred_class))
print(len(y_pred_class))
print(len(y_test))
print('---------------')'''

acc = accuracy_score(y_test, y_pred_class) * 100
print("Accuracy is--->", acc)

859/859 [==============================] - 52s 60ms/step
Accuracy is---> 94.37163244502695
